In [5]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


def simple_gradient():
    # print the gradient of 2x^2 + 5x
    x = Variable(torch.ones(2, 2) * 2, requires_grad=True)
    z = 2 * (x * x) + 5 * x
    # run the backpropagation
    z.backward(torch.ones(2, 2))
    print(x.grad)


def create_nn(batch_size=200, learning_rate=0.01, epochs=10,
              log_interval=10):

    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
        batch_size=batch_size, shuffle=True)

    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.fc1 = nn.Linear(28 * 28, 200)
            self.fc2 = nn.Linear(200, 200)
            self.fc3 = nn.Linear(200, 10)

        def forward(self, x):
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
            return F.log_softmax(x)

    net = Net()
    print(net)

    # create a stochastic gradient descent optimizer
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
    # create a loss function
    criterion = nn.NLLLoss()

    # run the main training loop
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = Variable(data), Variable(target)
            # resize data from (batch_size, 1, 28, 28) to (batch_size, 28*28)
            data = data.view(-1, 28*28)
            optimizer.zero_grad()
            net_out = net(data)
            loss = criterion(net_out, target)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                           100. * batch_idx / len(train_loader), loss.data[0]))

    # run a test loop
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        data = data.view(-1, 28 * 28)
        net_out = net(data)
        # sum up batch loss
        test_loss += criterion(net_out, target).data[0]
        pred = net_out.data.max(1)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


if __name__ == "__main__":
    run_opt = 2
    if run_opt == 1:
        simple_gradient()
    elif run_opt == 2:
        create_nn()

Net(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
)
Train Epoch: 0 [0/60000 (0%)]	Loss: 2.316136


/home/kriti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/home/kriti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:70: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 0 [2000/60000 (3%)]	Loss: 2.183335
Train Epoch: 0 [4000/60000 (7%)]	Loss: 1.890050
Train Epoch: 0 [6000/60000 (10%)]	Loss: 1.335527
Train Epoch: 0 [8000/60000 (13%)]	Loss: 0.803522
Train Epoch: 0 [10000/60000 (17%)]	Loss: 0.592661
Train Epoch: 0 [12000/60000 (20%)]	Loss: 0.464143
Train Epoch: 0 [14000/60000 (23%)]	Loss: 0.413233
Train Epoch: 0 [16000/60000 (27%)]	Loss: 0.454169
Train Epoch: 0 [18000/60000 (30%)]	Loss: 0.365753
Train Epoch: 0 [20000/60000 (33%)]	Loss: 0.350188
Train Epoch: 0 [22000/60000 (37%)]	Loss: 0.533101
Train Epoch: 0 [24000/60000 (40%)]	Loss: 0.303207
Train Epoch: 0 [26000/60000 (43%)]	Loss: 0.375205
Train Epoch: 0 [28000/60000 (47%)]	Loss: 0.330805
Train Epoch: 0 [30000/60000 (50%)]	Loss: 0.394054
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.173555
Train Epoch: 0 [34000/60000 (57%)]	Loss: 0.347506
Train Epoch: 0 [36000/60000 (60%)]	Loss: 0.240594
Train Epoch: 0 [38000/60000 (63%)]	Loss: 0.322175
Train Epoch: 0 [40000/60000 (67%)]	Loss: 0.387908
Train 

Train Epoch: 5 [34000/60000 (57%)]	Loss: 0.094397
Train Epoch: 5 [36000/60000 (60%)]	Loss: 0.079638
Train Epoch: 5 [38000/60000 (63%)]	Loss: 0.089295
Train Epoch: 5 [40000/60000 (67%)]	Loss: 0.029642
Train Epoch: 5 [42000/60000 (70%)]	Loss: 0.067258
Train Epoch: 5 [44000/60000 (73%)]	Loss: 0.050084
Train Epoch: 5 [46000/60000 (77%)]	Loss: 0.054902
Train Epoch: 5 [48000/60000 (80%)]	Loss: 0.075954
Train Epoch: 5 [50000/60000 (83%)]	Loss: 0.072566
Train Epoch: 5 [52000/60000 (87%)]	Loss: 0.060436
Train Epoch: 5 [54000/60000 (90%)]	Loss: 0.065930
Train Epoch: 5 [56000/60000 (93%)]	Loss: 0.097866
Train Epoch: 5 [58000/60000 (97%)]	Loss: 0.049429
Train Epoch: 6 [0/60000 (0%)]	Loss: 0.046417
Train Epoch: 6 [2000/60000 (3%)]	Loss: 0.089776
Train Epoch: 6 [4000/60000 (7%)]	Loss: 0.083492
Train Epoch: 6 [6000/60000 (10%)]	Loss: 0.080422
Train Epoch: 6 [8000/60000 (13%)]	Loss: 0.083720
Train Epoch: 6 [10000/60000 (17%)]	Loss: 0.118785
Train Epoch: 6 [12000/60000 (20%)]	Loss: 0.073022
Train Epoch

/home/kriti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:76: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/kriti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:80: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



Test set: Average loss: 0.0004, Accuracy: 9778/10000 (97%)

